In [59]:
import pandas as pd
%pip install requests
import requests
# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head(100)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-11T17:36:10.070Z,33.186501,-115.553001,7.5000,2.01,ml,14.0,224.0,0.094610,0.10,...,2025-11-11T17:38:14.471Z,"7 km SSW of Niland, CA",earthquake,0.86,1.300000,0.506,34.0,automatic,ci,ci
1,2025-11-11T17:26:34.799Z,32.364000,-102.081000,8.8137,1.20,ml,12.0,59.0,0.000000,0.40,...,2025-11-11T17:29:12.126Z,"37 km NW of Stanton, Texas",earthquake,0.00,2.797230,0.200,9.0,automatic,tx,tx
2,2025-11-11T17:10:08.350Z,32.063000,-101.676000,1.8428,1.30,ml,14.0,86.0,0.100000,0.70,...,2025-11-11T17:13:14.998Z,"12 km SE of Stanton, Texas",earthquake,0.00,1.841656,0.200,9.0,automatic,tx,tx
3,2025-11-11T17:08:54.850Z,36.470665,-121.088837,6.0300,1.33,md,17.0,58.0,0.029240,0.26,...,2025-11-11T17:27:18.587Z,"8 km SE of Pinnacles, CA",earthquake,0.56,1.700000,0.190,12.0,automatic,nc,nc
4,2025-11-11T16:35:50.930Z,38.832001,-122.817337,1.7100,0.75,md,7.0,71.0,0.015630,0.00,...,2025-11-11T16:52:17.416Z,"8 km NW of The Geysers, CA",earthquake,0.35,1.140000,0.170,8.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-11-11T03:44:28.924Z,59.460100,-151.381400,52.7000,1.90,ml,NaN,NaN,NaN,0.74,...,2025-11-11T03:48:20.988Z,"15 km E of Seldovia Village, Alaska",earthquake,NaN,0.500000,NaN,NaN,automatic,ak,ak
96,2025-11-11T03:35:09.463Z,57.217800,-156.494400,14.2000,2.50,ml,NaN,NaN,NaN,0.84,...,2025-11-11T03:37:05.612Z,"63 km ESE of Ugashik, Alaska",earthquake,NaN,1.100000,NaN,NaN,automatic,ak,ak
97,2025-11-11T03:28:37.114Z,13.847100,-58.461600,10.0000,5.00,mww,73.0,59.0,2.760000,1.09,...,2025-11-11T14:38:31.716Z,"134 km NE of Crane, Barbados",earthquake,8.82,1.885000,0.093,11.0,reviewed,us,us
98,2025-11-11T03:24:39.120Z,38.821335,-122.847000,1.6700,1.07,md,13.0,80.0,0.009518,0.02,...,2025-11-11T04:52:20.464Z,"9 km WNW of The Geysers, CA",earthquake,0.27,0.410000,0.150,14.0,automatic,nc,nc


In [60]:
data["country"] = data["place"].str.split(",").str[-1]
ranking = data["country"].value_counts().head(10)

ranking_df = ranking.reset_index()
ranking_df.columns = ["country", "count"]
ranking_df.index += 1

ranking_df["point"] = ranking_df["count"].rank(method="min", ascending=False).astype(int)
ranking_df["point"] = ranking_df["point"].max() - ranking_df["point"] + 1

ranking_mag = data.sort_values("mag", ascending=False)[["mag", "country"]].head(10)
ranking_mag = ranking_mag.reset_index(drop=True)
ranking_mag.index += 1
ranking_mag["point"] = ranking_mag.index.map(lambda x: 11 - x)

print(ranking_df,end="\n\n")
print(ranking_mag)

        country  count  point
1            CA   2349     10
2        Alaska   1918      9
3         Texas    578      8
4    Washington    195      7
5        Hawaii    191      6
6    New Mexico    170      5
7      Oklahoma    166      4
8       Montana    153      3
9        Russia    147      2
10       Nevada    140      1

    mag        country  point
1   6.8          Japan     10
2   6.5      Indonesia      9
3   6.5     Guadeloupe      8
4   6.4      Banda Sea      7
5   6.4          Japan      6
6   6.3    Afghanistan      5
7   6.3  Drake Passage      4
8   6.2    Timor Leste      3
9   6.1         Russia      2
10  6.0          Japan      1
